# Targeted Maximum Likelihood Estimator
The targeted maximum likelihood estimator (TMLE) is a doubly robust estimator. What distinguishes it from other doubly robust estimators (augmented-IPTW) is that it uses a secondary targeting (hence the name) step that optimizes the bias-variance tradeoff for the target parameter. A common target parameter is the sample average treatment effect, which compares to counterfactual where all individuals in the study sample were treated versus the counterfactual where no individual was treated. Throughout this section, I will use the notation that is common in the TMLE literature. This is a little different from the notation in other documents.

The target parameter is defined as
$$\psi = E_L\left[E\left[Y^{a=1}|L\right] - E\left[Y^{a=0}|L\right]\right]$$
We will focus on this parameter, but other estimates (like risk ratio and odds ratio) are also implemented

This tutorial focuses on estimating TMLE with missing outcome data. I will demonstrate with both the default regression models and machine learning algorithms.

## Missing Outcome Data
To motivate our example, we will use a simulated data set included with *zEpid*. In the data set, we have a cohort of HIV-positive individuals. We are interested in the sample average treatment effect of antiretroviral therapy (ART) on all-cause mortality at 45-weeks. Based on substantive background knowledge, we believe that the treated and untreated population are exchangeable based gender, age, CD4 T-cell count, and detectable viral load. 

While we conducted a complete-case analysis in the previous tutorial, TMLE can also natively handle missing outcome data. This is accomplished by using inverse probability of missing weights in the background. We can add this model by specifying the `missing_model()` function. 

To start, we will load our data and create some spline terms for age and CD4 T-cell count

In [1]:
from zepid import load_sample_data, spline
from zepid.causal.doublyrobust import TMLE

df = load_sample_data(False)
df[['age_rs1', 'age_rs2']] = spline(df, 'age0', n_knots=3, term=2, restricted=True)
df[['cd4_rs1', 'cd4_rs2']] = spline(df, 'cd40', n_knots=3, term=2, restricted=True)

dfcc = df.drop(columns=['cd4_wk45'])
dfcc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 547 entries, 0 to 546
Data columns (total 12 columns):
id         547 non-null int64
male       547 non-null int64
age0       547 non-null int64
cd40       547 non-null int64
dvl0       547 non-null int64
art        547 non-null int64
dead       517 non-null float64
t          547 non-null float64
age_rs1    547 non-null float64
age_rs2    547 non-null float64
cd4_rs1    547 non-null float64
cd4_rs2    547 non-null float64
dtypes: float64(6), int64(6)
memory usage: 55.6 KB


To accomodate missing outcome data, we proceed the previous process from the prior tutorial. However, this time we will also specify the `missing_model()` function. This model allows us to generate inverse probability of missing weights (IPMW) for the outcome. These weights are used to update the IPTW and the targeting process. Essentially, this procedure allows us to go from a missing completely at random assumption to the weaker assumption of missing at random. See Gruber, S., & van der Laan, M. J. (2011). tmle: An R package for targeted maximum likelihood estimation. for details on the procedure

To estimate TMLE using a parametric procedure, you can use the below code. In this example, we assume that outcome data is missing dependent on treatment (ART), gender, age, detected viral load, and CD4 T-cell count.

In [2]:
tml = TMLE(dfcc, exposure='art', outcome='dead')
tml.exposure_model('male + age0 + cd40 + cd4_rs1 + cd4_rs2 + dvl0', print_results=False)
tml.missing_model('art + male + age0 + cd40 + cd4_rs1 + cd4_rs2 + dvl0', print_results=False)
tml.outcome_model('art + male + age0 + cd40 + cd4_rs1 + cd4_rs2 + dvl0', print_results=False)
tml.fit()
tml.summary()

                Targeted Maximum Likelihood Estimator                 
Risk Difference:  -0.08
95.0% two-sided CI: (-0.153 , -0.008)
----------------------------------------------------------------------
Risk Ratio:  0.562
95.0% two-sided CI: (0.294 , 1.072)
----------------------------------------------------------------------
Odds Ratio:  0.512
95.0% two-sided CI: (0.247 , 1.058)


Looking back at the previous TMLE tutorial, our results are fairly consistent. 

## Maching Learning
As in the previous tutorial, we can also use machine learning to estimate the predicted values. Again, we will use super learner with neural network, naive Bayes, L1-penalized logistic regression, L2- penalized logistic regression, AdaBoost, and random forest. 

In [3]:
import supylearner
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB

# Initializing the ML algorithms
neural = MLPClassifier(hidden_layer_sizes=(8,), random_state=1463)
log1 = LogisticRegression(penalty='l1', random_state=201)
log2 = LogisticRegression(penalty='l2', random_state=103)
randf = RandomForestClassifier(random_state=141)
adaboost = AdaBoostClassifier(random_state=505)
bayes = GaussianNB()

# Initializing SuPyLearner with the above algorithms
lib = [neural, log1, log2, randf, adaboost, bayes]
libnames = ["Neural-Net", "Log_L1", "Log_L2", "Random Forest", "AdaBoost", "Bayes"]
sl = supylearner.SuperLearner(lib, libnames, loss="nloglik", K=10, print_results=False)

# Estimating TMLE with ML algorithms
tmle = TMLE(dfcc, exposure='art', outcome='dead')  # Step 1) Initialize TMLE class
tmle.exposure_model('male + age0 + age_rs1 + age_rs2 + cd40 + cd4_rs1 + cd4_rs2 + dvl0',
                    custom_model=sl, print_results=False)  # Step 2) Specify exposure model
tmle.missing_model('art + male + age0 + age_rs1 + age_rs2 + cd40 + cd4_rs1 + cd4_rs2 + dvl0',
                    custom_model=sl, print_results=False)  # Step 3) Specify exposure model
tmle.outcome_model('art + male + age0 + age_rs1 + age_rs2 + cd40 + cd4_rs1 + cd4_rs2 + dvl0',
                   custom_model=sl, print_results=False)  # Step 4) Specify outcome model
tmle.fit()  # Step 5) Targeting step
tmle.summary()

                Targeted Maximum Likelihood Estimator                 
Risk Difference:  -0.08
95.0% two-sided CI: (-0.15 , -0.009)
----------------------------------------------------------------------
Risk Ratio:  0.561
95.0% two-sided CI: (0.299 , 1.055)
----------------------------------------------------------------------
Odds Ratio:  0.512
95.0% two-sided CI: (0.252 , 1.04)


Again, we obtained similar results between the parametric TMLE and TMLE with super learner

# Conclusion
In this tutorial, I have described how TMLE can be estimated with missing outcome data and how to do so with *zEpid*. `TMLE` can be used to estimate missing data for binary or continuous outcomes. Please refer to other tutorials for more options and specifications available with `TMLE`.

## References

Schuler, Megan S., and Sherri Rose. "Targeted maximum likelihood estimation for causal inference in observational studies." American Journal of Epidemiology 185.1 (2017): 65-73.

van der Laan, Mark J., and Sherri Rose. Targeted learning: causal inference for observational and experimental data. Springer Science & Business Media, 2011.

van Der Laan, Mark J., and Daniel Rubin. "Targeted maximum likelihood learning." The International Journal of Biostatistics 2.1 (2006).

Gruber, S., & van der Laan, M. J. (2011). tmle: An R package for targeted maximum likelihood estimation.